---
title: STEM post processing
authors: [Toma Susi]
date: 2024/07/24
---

In [ ]:
import abtem
stacked_measurements = abtem.from_zarr("data/STO_LTO_STEM.zarr").compute()

In [ ]:
interpolated_measurements = stacked_measurements.interpolate(0.05)

interpolated_measurements.show(explode=True);

In [ ]:
blurred_measurements = interpolated_measurements.gaussian_filter(0.35)

blurred_measurements.show(explode=True);

In [ ]:
noisy_measurements = blurred_measurements.poisson_noise(dose_per_area=1e5, seed=100)

noisy_measurements.show(explode=True, figsize=(12, 4));

In [ ]:
processed_measurements = abtem.stack([interpolated_measurements, blurred_measurements, noisy_measurements],
                               ("Interpolated", "Blurred", "Noised"))

In [ ]:
from scipy.ndimage import zoom

scalex = interpolated_measurements[0].shape[0]/stacked_measurements[0].array.shape[0]
scaley = interpolated_measurements[1].shape[1]/stacked_measurements[0].array.shape[1]

zoomed0 = zoom(stacked_measurements[0].array, (scalex, scaley), order=0)
zoomed1 = zoom(stacked_measurements[1].array, (scalex, scaley), order=0)
zoomed2 = zoom(stacked_measurements[2].array, (scalex, scaley), order=0)

In [ ]:
from abtem import Images

image0 = Images(zoomed0, sampling = interpolated_measurements[0].sampling)
image1 = Images(zoomed1, sampling = interpolated_measurements[1].sampling)
image2 = Images(zoomed2, sampling = interpolated_measurements[2].sampling)

zoomed_measurements = abtem.stack([image0, image1, image2], ("BF", "MAADF", "HAADF"))

In [ ]:
all_measurements = abtem.stack([zoomed_measurements, interpolated_measurements, blurred_measurements, noisy_measurements],
                               ("Nyquist-sampled", "Interpolated", "Blurred", "Noised"))

all_measurements.to_zarr("data/processed_STEM.zarr")

In [2]:
%matplotlib ipympl

abtem.config.set({"visualize.cmap": "viridis"})
abtem.config.set({"visualize.continuous_update": True})
abtem.config.set({"visualize.autoscale": True})

all_measurements = abtem.from_zarr("data/processed_STEM.zarr")

In [12]:
#| label: app:stem_processing
viz = all_measurements.show(interact=True, explode=(1,), figsize=(5,5))

[########################################] | 100% Completed | 103.55 ms


ImageGUI(children=(VBox(children=(SelectionSlider(options=('Nyquist-sampled', 'Interpolated', 'Blurred', 'Nois…